In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:86% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.output {font-size:15pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:15px;}
</style>
"""))

# 1주일 내용

In [2]:
import pymysql
from dotenv import load_dotenv
import os
import pandas as pd
from datetime import datetime, timedelta

def DBMS_last_year_one_week() :
    '''내일을 기준으로 작년 1주일치 생활인구수와 대여량을 DataFrame으로 반환하는 함수'''
    load_dotenv()
    host = os.getenv('host')
    user = os.getenv('user')
    password = os.getenv('password')
    database = os.getenv('database')
    port = int(os.getenv('port', 3306))

    # 현재 시각
    now = datetime.now()

    # 1주일치 시간 list
    this_week = [now + timedelta(days=i) for i in range(1,8)]

    # 1년 전 1주일치 데이터
    dt_1y = [day.replace(year=day.year - 1) for day in this_week]

    # DB 컬럼명에 맞게 쿼리 작성 (연, 월, 일, 시)

    total_last_week = pd.DataFrame(columns = ['연도','월','일','시','행정구','총생활인구수_1년전','대여량_1년전'])
     # total_last_week = pd.DataFrame(columns = ['year','month','day','hour','district','total_population','rental_count'])
    for data in dt_1y :
        query =f"""
            SELECT *
            FROM basic_data
            WHERE year = {data.year}
              AND month = {data.month}
              AND day = {data.day}
              """

        try:
            connection = pymysql.connect(
                host=host,
                user=user,
                password=password,
                database=database,
                port=port,
                charset='utf8mb4',
                cursorclass=pymysql.cursors.DictCursor
            )
            with connection.cursor() as cursor:
                cursor.execute(query)
                data = cursor.fetchall()
                whole_data = pd.DataFrame(data)
                using_data = whole_data[['year','month','day','hour','district','total_population','rental_count']]
                using_data.columns = total_last_week.columns
                total_last_week = pd.concat([total_last_week, using_data],axis=0)
        except Exception as e:
            print('에러 발생:', e)
        finally:
            try:
                connection.close()
            except:
                pass
    return total_last_week # 일주일 치 연도, 월, 일, 시, 행정구, 총생활인구수_1년전, 대여량_1년전

In [5]:
import requests
import pandas as pd

def get_hourly_weather_seoul_openmeteo():
    SEOUL_DISTRICTS = [
        {"name": "강남구", "lat": 37.5172, "lon": 127.0473},
        {"name": "강동구", "lat": 37.5301, "lon": 127.1238},
        {"name": "강북구", "lat": 37.6396, "lon": 127.0256},
        {"name": "강서구", "lat": 37.5509, "lon": 126.8495},
        {"name": "관악구", "lat": 37.4784, "lon": 126.9516},
        {"name": "광진구", "lat": 37.5384, "lon": 127.0823},
        {"name": "구로구", "lat": 37.4955, "lon": 126.8878},
        {"name": "금천구", "lat": 37.4604, "lon": 126.9006},
        {"name": "노원구", "lat": 37.6542, "lon": 127.0568},
        {"name": "도봉구", "lat": 37.6688, "lon": 127.0472},
        {"name": "동대문구", "lat": 37.5744, "lon": 127.0396},
        {"name": "동작구", "lat": 37.5124, "lon": 126.9392},
        {"name": "마포구", "lat": 37.5663, "lon": 126.9014},
        {"name": "서대문구", "lat": 37.5791, "lon": 126.9368},
        {"name": "서초구", "lat": 37.4836, "lon": 127.0327},
        {"name": "성동구", "lat": 37.5633, "lon": 127.0364},
        {"name": "성북구", "lat": 37.5894, "lon": 127.0167},
        {"name": "송파구", "lat": 37.5146, "lon": 127.1056},
        {"name": "양천구", "lat": 37.5179, "lon": 126.8666},
        {"name": "영등포구", "lat": 37.5264, "lon": 126.8962},
        {"name": "용산구", "lat": 37.5323, "lon": 126.9909},
        {"name": "은평구", "lat": 37.6027, "lon": 126.9291},
        {"name": "종로구", "lat": 37.5735, "lon": 126.9790},
        {"name": "중구", "lat": 37.5636, "lon": 126.9977},
        {"name": "중랑구", "lat": 37.6065, "lon": 127.0927}
    ]

    result_rows = []
    for gu in SEOUL_DISTRICTS:
        latitude, longitude, name = gu["lat"], gu["lon"], gu["name"]
        url = (
            f"https://api.open-meteo.com/v1/forecast"
            f"?latitude={latitude}&longitude={longitude}"
            "&hourly=temperature_2m,precipitation,weathercode,relative_humidity_2m,wind_speed_10m"
            "&forecast_days=8&timezone=Asia%2FSeoul"
        )
        resp = requests.get(url)
        if resp.status_code == 200:
            json_data = resp.json()
            if 'hourly' not in json_data:
                print(f"[경고] {name} 데이터에 'hourly' 없음. 응답: {json_data}")
                continue
            data = json_data['hourly']
            for t, temp, rain, code, rh, wind in zip(
                data['time'],
                data['temperature_2m'],
                data['precipitation'],
                data['weathercode'],
                data['relative_humidity_2m'],
                data['wind_speed_10m'],
            ):
                result_rows.append({
                    "행정구": name,
                    "일시": t,
                    "강수": rain,
                    "습도": rh,
                    "풍속": wind,
                    "기온": temp,
                })
        else:
            print(f"[오류] {name} API 호출 실패: {resp.status_code}")

    # DataFrame 생성
    df = pd.DataFrame(result_rows)

    # 일시 → datetime 변환 및 파생 변수 추출
    df['일시'] = pd.to_datetime(df['일시'])
    df['연'] = df['일시'].dt.year
    df['월'] = df['일시'].dt.month
    df['일'] = df['일시'].dt.day
    df['시'] = df['일시'].dt.hour
    df['날짜'] = df['일시'].dt.date  # 날짜만 필요할 때

    return df

In [6]:
weather = get_hourly_weather_seoul_openmeteo()
weather # 행정구, 일시, 강수, 습도, 풍속, 기온, 연, 월, 일, 시

,행정구,일시,강수,습도,풍속,기온,연,월,일,시,날짜
0,강남구,2025-07-03 00:00:00,0.0,99,3.8,24.0,2025,7,3,0,2025-07-03
1,강남구,2025-07-03 01:00:00,0.0,99,2.1,24.0,2025,7,3,1,2025-07-03
2,강남구,2025-07-03 02:00:00,0.0,99,2.5,23.9,2025,7,3,2,2025-07-03
3,강남구,2025-07-03 03:00:00,0.0,99,4.7,23.9,2025,7,3,3,2025-07-03
4,강남구,2025-07-03 04:00:00,0.0,99,2.2,23.8,2025,7,3,4,2025-07-03
...,...,...,...,...,...,...,...,...,...,...,...
4795,중랑구,2025-07-10 19:00:00,0.0,37,15.0,34.0,2025,7,10,19,2025-07-10
4796,중랑구,2025-07-10 20:00:00,0.0,40,10.0,32.1,2025,7,10,20,2025-07-10
4797,중랑구,2025-07-10 21:00:00,0.0,43,6.3,30.5,2025,7,10,21,2025-07-10
4798,중랑구,2025-07-10 22:00:00,0.0,45,5.2,29.4,2025,7,10,22,2025-07-10


# 7일간의 자전거 수요를 예측하는 함수

In [12]:
import pandas as pd
import numpy as np
import joblib
from datetime import datetime, timedelta
import holidays
import tensorflow as tf

# 예측 함수
def predict_bike_demand():
    """
    내일부터 7일간의 자전거 수요를 예측하는 함수, 강남구, 강동구....중구, 중랑구 순으로 0시부터 24시 까지의 데이터를 순서대로 반환
    """
    le = joblib.load('ENCODER_SCALER/labelencoders.pkl')
    scaler = joblib.load('ENCODER_SCALER/scaler.pkl')
    feature_cols = joblib.load('ENCODER_SCALER/featurecols.pkl')
    num_cols = joblib.load('ENCODER_SCALER/numcols.pkl')
    model = tf.keras.models.load_model('best_dnn_model_lr.h5')

    df_1y = DBMS_last_year_one_week()[['월', '일', '시', '행정구', '총생활인구수_1년전', '대여량_1년전']] # 일주일 치 연도, 월, 일, 시, 행정구, 총생활인구수_1년전, 대여량_1년전
    weather_df = get_hourly_weather_seoul_openmeteo()[['행정구', '월', '일', '시','강수', '습도', '풍속', '기온','일시']]
    weather_df['일시'] = pd.to_datetime(weather_df['일시'])
    df_total = weather_df.merge(df_1y, on=['행정구','월','일','시'])
    
    kr_holidays = holidays.KR(years=[datetime.now().year, datetime.now().year+1])
    df_total['요일'] = df_total['일시'].dt.weekday
    df_total['주말구분'] = (df_total['요일'] >= 5).astype(int)
    df_total['공휴일'] = df_total['일시'].dt.date.isin(kr_holidays).astype(int)

#     input_df = pd.DataFrame([input_dict])
    df_total['행정구'] = le.transform(df_total['행정구'].values)
    df_total[num_cols] = scaler.transform(df_total[num_cols])
    X_input = df_total[feature_cols].astype('float32').values
    pred = model.predict(X_input,verbose=0).round().astype(np.int32)
    return pred

In [6]:
datas = predict_bike_demand()

In [8]:
datas

array([[134],
       [116],
       [ 83],
       ...,
       [327],
       [276],
       [209]])

# 결과를 df로 함수화하기

In [9]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
def result_df_return(datas) :
    districts = ['강남구',"강동구","강북구", "강서구","관악구","광진구","구로구","금천구","노원구","도봉구","동대문구","동작구","마포구","서대문구","서초구","성동구","성북구","송파구","양천구","영등포구","용산구","은평구","종로구","중구","중랑구"]
    
    base_date = datetime.now()
    num_days = 7
    num_hours = 24
    all_dates = [base_date + timedelta(days=1+i) for i in range(num_days)]

    date_hour_list = []
    for d in all_dates:
        for h in range(num_hours):
            date_hour_list.append((d.date(), h))

    # datas shape 자동 보정
    datas = np.array(datas).reshape(len(districts), len(date_hour_list))  
    
    df = pd.DataFrame(datas, index=districts, columns=pd.MultiIndex.from_tuples(date_hour_list, names=['날짜', '시']))

    df_long = df.stack(level=[0, 1]).reset_index()
    df_long.columns = ['행정구', '날짜', '시', '예측값']

#     print(df_long.head())
#     print(df_long.shape)  # (3600, 4) => 현재는 7일치로, 4200,4
    return df_long


In [10]:
df_long = result_df_return(datas)

In [11]:
df_long[df_long['행정구']=='관악구'].head(60)

,행정구,날짜,시,예측값
672,관악구,2025-07-06,0,165
673,관악구,2025-07-06,1,123
674,관악구,2025-07-06,2,98
675,관악구,2025-07-06,3,75
676,관악구,2025-07-06,4,11
677,관악구,2025-07-06,5,18
678,관악구,2025-07-06,6,27
679,관악구,2025-07-06,7,35
680,관악구,2025-07-06,8,77
681,관악구,2025-07-06,9,143


In [63]:
district = ['강남구',"강동구","강북구", "강서구","관악구","광진구","구로구","금천구","노원구","도봉구","동대문구","동작구","마포구","서대문구","서초구","성동구","성북구","송파구","양천구","영등포구","용산구","은평구","종로구","중구","중랑구"]
for gu in district :
    print(gu,"의 1일간 대여")
    display(df_long[df_long['행정구']==gu].head(24))

강남구 의 1일간 대여


,행정구,날짜,시,예측값
0,강남구,2025-07-03,0,105
1,강남구,2025-07-03,1,91
2,강남구,2025-07-03,2,51
3,강남구,2025-07-03,3,55
4,강남구,2025-07-03,4,37
5,강남구,2025-07-03,5,70
6,강남구,2025-07-03,6,162
7,강남구,2025-07-03,7,332
8,강남구,2025-07-03,8,511
9,강남구,2025-07-03,9,300


강동구 의 1일간 대여


,행정구,날짜,시,예측값
168,강동구,2025-07-03,0,171
169,강동구,2025-07-03,1,104
170,강동구,2025-07-03,2,65
171,강동구,2025-07-03,3,41
172,강동구,2025-07-03,4,47
173,강동구,2025-07-03,5,135
174,강동구,2025-07-03,6,249
175,강동구,2025-07-03,7,577
176,강동구,2025-07-03,8,628
177,강동구,2025-07-03,9,338


강북구 의 1일간 대여


...

,행정구,날짜,시,예측값
3696,종로구,2025-07-03,0,63
3697,종로구,2025-07-03,1,62
3698,종로구,2025-07-03,2,45
3699,종로구,2025-07-03,3,29
3700,종로구,2025-07-03,4,36
3701,종로구,2025-07-03,5,65
3702,종로구,2025-07-03,6,129
3703,종로구,2025-07-03,7,302
3704,종로구,2025-07-03,8,472
3705,종로구,2025-07-03,9,260


중구 의 1일간 대여


,행정구,날짜,시,예측값
3864,중구,2025-07-03,0,54
3865,중구,2025-07-03,1,41
3866,중구,2025-07-03,2,43
3867,중구,2025-07-03,3,43
3868,중구,2025-07-03,4,33
3869,중구,2025-07-03,5,30
3870,중구,2025-07-03,6,70
3871,중구,2025-07-03,7,199
3872,중구,2025-07-03,8,384
3873,중구,2025-07-03,9,202


중랑구 의 1일간 대여


,행정구,날짜,시,예측값
4032,중랑구,2025-07-03,0,134
4033,중랑구,2025-07-03,1,108
4034,중랑구,2025-07-03,2,74
4035,중랑구,2025-07-03,3,63
4036,중랑구,2025-07-03,4,44
4037,중랑구,2025-07-03,5,81
4038,중랑구,2025-07-03,6,146
4039,중랑구,2025-07-03,7,307
4040,중랑구,2025-07-03,8,396
4041,중랑구,2025-07-03,9,234


# 위 내용을 토대로 M2모델 설계

In [13]:
df_long # 행정구 날짜 시 예측값

,행정구,날짜,시,예측값
0,강남구,2025-07-06,0,134
1,강남구,2025-07-06,1,116
2,강남구,2025-07-06,2,83
3,강남구,2025-07-06,3,62
4,강남구,2025-07-06,4,13
...,...,...,...,...
4195,중랑구,2025-07-12,19,380
4196,중랑구,2025-07-12,20,324
4197,중랑구,2025-07-12,21,327
4198,중랑구,2025-07-12,22,276


In [14]:
df_PM = pd.read_csv('PM_data/서울시_시간별_PM_배치_2021_2025.csv')
df_PM

,일시,자치구,시간별_PM_배치수
0,2021-01-01 00:00:00,강남,24
1,2021-01-01 01:00:00,강남,24
2,2021-01-01 02:00:00,강남,24
3,2021-01-01 03:00:00,강남,24
4,2021-01-01 04:00:00,강남,24
...,...,...,...
832651,2025-12-31 19:00:00,은평구,77
832652,2025-12-31 20:00:00,은평구,53
832653,2025-12-31 21:00:00,은평구,53
832654,2025-12-31 22:00:00,은평구,53
